In [2]:
import pandas as pd
import numpy as np

from keras.layers.recurrent import LSTM
from keras.models import Sequential

# Neurone Normale

In [98]:
class Neurone(object):
    def __init__(self,  w=1):
        self.w = w
    
    def compute(self, x):
        return max(0, self.w * x)

# Neurone Requent  (RNN)
## $ S = relu(h_{t+1}) $

## $ h_{t+1} = relu(\theta * X_t + w * h_t) $

In [3]:
def relu(x):
    return max(0, x)

class RecurenceNeurone(object):
    # constructeur
    def __init__(self, theta=1, w=1, h=0):
        self.theta = theta # poids
        self.w = w
        self.h = h # memoir interne
        self.avtivationRelu = relu #on déclare que notre fonction d'activation est la relu
        self.outputFunc = relu #on déclare que notre fonction de sortie est une relu
    
    # methode    
    def compute(self, x):
        z = self.theta * x + self.w * self.h
        self.h = self.avtivationRelu(z)
        return self.outputFunc(self.h)
        
        

In [4]:
neurone1 = RecurenceNeurone() #creation d'un neurone qui avance de 1 en 1
neurone1.compute(1)

1

In [5]:
neurone1.compute(1)

2

In [6]:
for i in range(15):
    print(neurone1.compute(1))

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


# Long Short Term Memory (LSTM)
## Step by step
### Input gate $ i_t = \sigma(W_{i} * [h_{t-1}, x_t] + b_i)$   permet de savoir si on ecrit dans c

## $\tilde{c}_t = \tanh(W_{ci} * [h_{t-1}, x_t] + b_c)$

### Forget gate $ f_t = \sigma (W_{f} * [h_{t-1}, x_t] + b_f)$  permet de savoir si on oublie cequ'il y avait dans c

### Cell state $ c_t = f_t * c_{t-1} + i_t * \tilde{c}_t$    

### Output gate $ o_t = \sigma (W_{ox} * [h_{t-1}, x_t] + b_o)$  

###  Hidden state $ ht=o_t * tanh(c_t) $   

In [107]:
def sigmoid(x):
    return 1.0/(np.exp(-x) + 1.0)
    
class LSTM_Neurone(object):
    # Constructeur
    def __init__(self):
        self.h = np.array([0])# initialisation de h à 0
        self.c = 0 # initialisation de la memoire à 0
        
        #initialisation des theta de i, forgate gate, c et de l'output à 1
        self.w_i = np.array([1, 1])
        self.w_f = np.array([1, 1])
        self.w_c = np.array([1, 1])
        self.w_o = np.array([1, 1])
        
        #on initialise les biais de i, forgate gate, c et de l'output à 0.1
        self.b_i = 0.5
        self.b_f = 0.5
        self.b_c = 0.5
        self.b_o = 0.5
        
        self.activation = relu
        self.outputFunc = sigmoid
    
    # Methode    
    def compute(self, x):
        input_gate = self.activation( self.w_i.dot(np.concatenate([self.h, x]))+ self.b_i)
        forget_gate = self.activation( self.w_f.dot(np.concatenate([self.h, x])) + self.b_f)
        c_tilde =  np.tanh( self.w_c.dot(np.concatenate([self.h, x])) + self.b_c)
        
        self.o = self.outputFunc(self.w_o.dot(np.concatenate([self.h, x])) + self.b_o)
            
        self.c = forget_gate * self.c + input_gate * c_tilde
        #print("C = ", self.c)
        #print("C_tilde = ", c_tilde)
        self.h = np.array([self.o * np.tanh(self.c)])
        #print("h = ", self.h )
        return self.o
        

In [108]:
nLSTM = LSTM_Neurone() #Creation d'un neurone
print(nLSTM.compute(np.array([1])))
print()
print(nLSTM.compute(np.array([0])))
print()
print(nLSTM.compute(np.array([1])))
print()
print(nLSTM.compute(np.array([1])))
print()
print(nLSTM.compute(np.array([1])))
print()
print(nLSTM.compute(np.array([0])))

0.8175744761936437

0.7713736452486172

0.9058531692586362

0.9172725402029648

0.9181349737426983

0.8050458726217169


## Variante GRU

## $ c_{t+1} = f_t * c_t + (1 - f_t) * \tilde{c}_t $

In [109]:
class GRU_Neurone(object):
    # Constructeur
    def __init__(self):
        self.h = np.array([0])# initialisation de h à 0
        self.c = 0 # initialisation de la memoire à 0
        
        #initialisation des theta de i, forgate gate, c et de l'output à 1
        self.w_f = np.array([1, 1])
        self.w_c = np.array([1, 1])
        self.w_o = np.array([1, 1])
        
        #on initialise les biais de i, forgate gate, c et de l'output à 0.5
        self.b_f = 0.5
        self.b_c = 0.5
        self.b_o = 0.5
        
        self.activation = relu
        self.outputFunc = sigmoid
    
    # Methode    
    def compute(self, x):
        forget_gate = self.activation( self.w_f.dot(np.concatenate([self.h, x])) + self.b_f)
        c_tilde = np.tanh( self.w_c.dot(np.concatenate([self.h, x])) + self.b_c)
        
        self.o = self.outputFunc(self.w_o.dot(np.concatenate([self.h, x])) + self.b_o)
        
        self.c = forget_gate * self.c + (1 - forget_gate) * c_tilde
        self.h = np.array([self.o * np.tanh(self.c)])
        return self.o

In [110]:
nGRU = GRU_Neurone() #Creation d'un neurone
print(nGRU.compute(np.array([1])))
print()
print(nGRU.compute(np.array([0])))
print()
print(nGRU.compute(np.array([1])))
print()
print(nGRU.compute(np.array([1])))
print()
print(nGRU.compute(np.array([1])))
print()
print(nGRU.compute(np.array([0])))

0.8175744761936437

0.5382595861413569

0.8222908789205312

0.7671334523426895

0.7450758037141454

0.5087422726465187
